**Manipulate MEP data for postprocessing analysis**

Adrian Wiegman

2023-08-12

-------

## Description

## Setup Environment

In [68]:
# iphython options
# delete variables in workspace
%reset -f
#places plots inline
%matplotlib inline
#automatically reloads modules if they are changed
%load_ext autoreload 
%autoreload 2
# this codeblock sets up the environment from jupyter notebooks
setup_notebook = r"C:\Users\Adrian.Wiegman\Documents\GitHub\Wiegman_USDA_ARS\Cran_Q_C\2_gis\scripts\_Setup.ipynb"
%run $setup_notebook # magic command to run the notebook 

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
***
loading python modules...

  `module_list` contains names of all loaded modules

...module loading complete

***
loading user defined functions...

type `fn_`+TAB to for autocomplete suggestions

 the object `def_list` contains user defined function names:
   fn_get_info
   fn_arcgis_table_to_df
   fn_arcgis_table_to_np_to_pd_df
   fn_run_script_w_propy_bat
   fn_try_mkdir
   fn_hello
   fn_recursive_glob_search
   fn_regex_search_replace
   fn_regex_search_0
   fn_arcpy_table_to_excel
   fn_agg_sum_df_on_group
   fn_add_prefix_suffix_to_selected_cols
   fn_calc_pct_cover_within_groups
   fn_buildWhereClauseFromList
   fn_FA_to_Q
   fn_alter_field_double
   fn_return_float
   fn_classify_wetlands

 use ??{insert fn name} to inspect
 for example running `??fn_get_info` returns:
Signature: fn_get_info(name='fn_get_info')
Source:   
def fn_get_info(name='fn_get_info'):
    '''
    returns the sourc

In [112]:
# remove the wareham estuary surface from the watershed layer
_ = arcpy.management.SelectLayerByAttribute(
    in_layer_or_view="MEP_Subwatersheds_All_Copy",
    selection_type="NEW_SELECTION",
    where_clause="OBJECTID_12 = 834",
    invert_where_clause=True
)

# remove overlaps between warehame and buttermilk bay
_ = arcpy.analysis.RemoveOverlapMultiple(
    in_features=_,
    out_feature_class=r"C:\Workspace\Geodata\Cran_Q_C\Default.gdb\MEP_Subwatersheds_All_Copy_RemoveOverlapMultiple",
    method="THIESSEN",
    join_attributes="ALL"
)

# dissolve subwatersheds by embayment
_ = arcpy.management.Dissolve(
    in_features=_,
    out_feature_class=r"C:\Workspace\Geodata\Cran_Q_C\Default.gdb\MEP_Subwatersheds_All_Copy_Rem_Dissolve",
    dissolve_field="EMBAY_NAME",
    statistics_fields=None,
    multi_part="MULTI_PART",
    unsplit_lines="DISSOLVE_LINES",
    concatenation_separator=""
)

# clip to remove watersheds that fall outside of grounwater model domain
_ = arcpy.analysis.Clip(
    in_features=_,
    clip_features="domain_poly",
    out_feature_class=r"C:\Workspace\Geodata\Cran_Q_C\Default.gdb\MEP_Subwatersheds_All_Copy_Rem_Dis_Domain",
    cluster_tolerance=None
)

# buffer to remove holes in polygons
_ = arcpy.analysis.Buffer(
    in_features=_,
    out_feature_class=r"C:\Workspace\Geodata\Cran_Q_C\Default.gdb\MEP_Subwatersheds_All_Copy_Rem_Dis_Domain_Buff1",
    buffer_distance_or_field="6 Meters",
    line_side="FULL",
    line_end_type="ROUND",
    dissolve_option="NONE",
    dissolve_field=None,
    method="PLANAR")

In [113]:
# negative buffer to return polygons back to original area 
_ = arcpy.analysis.Buffer(
    in_features=_,
    out_feature_class=r"C:\Workspace\Geodata\Cran_Q_C\Default.gdb\MEP_Embayments_Domain",
    buffer_distance_or_field="-6 Meters",
    line_side="FULL",
    line_end_type="ROUND",
    dissolve_option="NONE",
    dissolve_field=None,
    method="PLANAR")

In [114]:
arcpy.management.CalculateField(
    in_table="MEP_Embayments_Domain",
    field="embayshed_area_m2",
    expression="!Shape_Area!",
    expression_type="PYTHON3",
    code_block="",
    field_type="DOUBLE",
    enforce_domains="NO_ENFORCE_DOMAINS"
)

<Result 'C:\\Workspace\\Geodata\\Cran_Q_C\\Default.gdb\\MEP_Embayments_Domain'>

In [ ]:
arcpy.conversion.TableToExcel(
    Input_Table="MEP_Embayments_Domain",
    Output_Excel_File=r"C:\Workspace\Geodata\Cran_Q_C\outputs\MEP_Embayments_Domain_TableToExcel.xlsx",
    Use_field_alias_as_column_header="NAME",
    Use_domain_and_subtype_description="CODE")

In [64]:
# write subembayments file with watershed MVP  nitrogen load data to excel for postprocessing
arcpy.conversion.TableToExcel(
    Input_Table=r"C:\Workspace\Geodata\Massachusetts\MEP\CC_Subembayments\Subembayments.shp",
    Output_Excel_File=r"C:\Workspace\Geodata\Cran_Q_C\outputs\Subembayments_TableToExcel.xlsx",
    Use_field_alias_as_column_header="NAME",
    Use_domain_and_subtype_description="CODE")

<Result 'C:\\Workspace\\Geodata\\Cran_Q_C\\outputs\\Subembayments_TableToExcel.xlsx'>